<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 200 kB 47.6 MB/s 
     |████████████████████████████████| 199 kB 49.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=efa8c0660641574285f850ce6cecbc556fe159c8271b62af5d363f4ba4c26fb0
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-17 20:31:56--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-17 20:32:22--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.48, 99.84.160.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236472510 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.52M  83.3MB/s    in 2.7s    

2022-11-17 20:32:25 (83.3 MB/s) - ‘name.basics.tsv.gz’ saved [236472510/236472510]

--2022-11-17 20:32:25--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.94, 99.84.160.48, 99.84.160.41, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287698034 (274M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.37M  34.1M

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00564...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [8]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
#df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw różnicę w różnych typach łączenia tabel df_name_basics_selected i df_title_principals_selected:

In [10]:
from timeit import default_timer as timer

start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53075170, 8)
joined table size:  (53069043, 12) time of joining tables:  171.74063872999977


In [11]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53075170, 8)
joined table size:  (53075170, 12) time of joining tables:  159.076253448


In [12]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53082998, 8)
joined table size:  (53076833, 12) time of joining tables:  159.27779396899996


In [13]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53082998, 8)
joined table size:  (53082998, 12) time of joining tables:  150.63972218800018


In [14]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53075170, 8)
joined table size:  (53069043, 12) time of joining tables:  152.7818813820004


In [15]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53075170, 8)
joined table size:  (53075170, 12) time of joining tables:  166.57404779100034


In [16]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53082998, 8)
joined table size:  (53076833, 12) time of joining tables:  151.34208891700018


In [17]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53082998, 8)
joined table size:  (53082998, 12) time of joining tables:  149.55194459300037


In [18]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

AnalysisException: ignored

In [ ]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

---------------------------------------------------------------------------------------------------

In [ ]:
df_name_basics = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

In [ ]:
start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

Tutaj za każdym razem wychodzą różne wyniki, dlatego ciężko stwierdzić, który sposób łączenia danych jest najszybszy, jednak zauważamy, że jedynie dwa sposoby ("inner" i "left") dają odpowiednie dane, gdyż otrzymujemy wtedy dane o aktorach z pełnymi danymi personalnymi bez zbędnych wierszy nieprzypisanych do żadnego imienia i nazwiska.

In [ ]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample